# PCA statistical arbitrage
Em dang test truoc tren S&P 500 roi sau do qua VN sau
Key idea, long some stock and short SPY
have a stock pool ( representing SPY) , use PCA on that to extract residuals, then apply ou method to see which one is undervalued ( in stat meaning) ( which has low s-score)
Buy them --> short SPY

In [36]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import yfinance as yf
import statsmodels.api as sm
from typing import List, Dict, Tuple

class PCA_StatArb_Strategy:
    def __init__(
        self,
        tickers: List[str],
        start_date: str,
        end_date: str,
        initial_balance: float = 1000000,
        fee_rate: float = 0.0043,  # 0.43% of trading value
        margin_spy: float = 0.25,  # 25% margin for SPY short
        pca_window: int = 252,
        ou_window: int = 60,
        n_factors: int = 15,
        allocation_per_stock: float = 0.05,
        beta_window: int = 252,
    ):
        self.tickers = tickers + ['SPY']
        self.start_date = start_date
        self.end_date = end_date
        self.initial_balance = initial_balance
        self.fee_rate = fee_rate
        self.margin_spy = margin_spy
        self.pca_window = pca_window
        self.ou_window = ou_window
        self.n_factors = n_factors
        self.allocation_per_stock = allocation_per_stock
        self.beta_window = beta_window

        # State variables
        self.cash = initial_balance
        self.positions: Dict[str, int] = {ticker: 0 for ticker in self.tickers}
        self.entry_prices: Dict[str, float] = {ticker: 0 for ticker in self.tickers}
        self.margin_posted = 0
        self.asset_list = []
        self.data = None
        self.returns = None
        self.betas = None
        self.residuals = None
        self.ou_params = None
        self.signals = None
        self.pca_metrics = {}  # Store PCA performance metrics

    def fetch_data(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Fetch and return closing prices and returns DataFrames for stocks and SPY."""
        data = yf.download(self.tickers, start=self.start_date, end=self.end_date)['Close']
        data = data.dropna()
        returns = data.pct_change().dropna()
        self.data = data
        self.returns = returns
        return data, returns

    def calculate_betas(self) -> pd.DataFrame:
        """Calculate and return rolling betas for each stock relative to SPY."""
        spy_returns = self.returns['SPY']
        betas = pd.DataFrame(index=self.returns.index, columns=[t for t in self.tickers if t != 'SPY'])
        for stock in betas.columns:
            stock_returns = self.returns[stock]
            rolling_cov = stock_returns.rolling(window=self.beta_window).cov(spy_returns)
            rolling_var_spy = spy_returns.rolling(window=self.beta_window).var()
            betas[stock] = rolling_cov / rolling_var_spy
        self.betas = betas.dropna()
        return self.betas

    def compute_pca_factors(self) -> Tuple[pd.Series, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
        """Compute PCA residuals, explained variance, loadings, and eigenvalues for stock returns (excluding SPY)."""
        returns_stocks = self.returns.drop(columns=['SPY'])  # Exclude SPY
        n = self.pca_window
        residuals = pd.DataFrame(index=returns_stocks.index, columns=returns_stocks.columns)
        explained_var = pd.Series(index=returns_stocks.index[n-1:], dtype=float)
        eigenvalues_df = pd.DataFrame(index=returns_stocks.index[n-1:], columns=[f'Eigen_{i+1}' for i in range(self.n_factors)])
        loadings_list = []

        for t in range(n, len(returns_stocks)):
            window = returns_stocks.iloc[t - n:t]
            pca = PCA(n_components=self.n_factors)
            pca.fit(window)
            factors = pca.transform(window)
            loadings = pca.components_.T  # Stock weights in each factor
            explained = np.dot(factors, loadings.T)
            residuals.iloc[t] = window.iloc[-1] - explained[-1]  # Residual for latest day
            explained_var.iloc[t - n] = sum(pca.explained_variance_ratio_)  # Cumulative explained variance
            eigenvalues_df.iloc[t - n] = pca.explained_variance_  # Eigenvalues
            loadings_df = pd.DataFrame(loadings, index=window.columns, columns=[f'Factor_{i+1}' for i in range(self.n_factors)])
            loadings_df['date'] = window.index[-1]
            loadings_list.append(loadings_df)

        residuals = residuals.dropna()
        loadings_all = pd.concat(loadings_list)
        self.residuals = residuals
        self.pca_metrics = {
            'explained_variance': explained_var,
            'loadings': loadings_all,
            'eigenvalues': eigenvalues_df
        }
        return explained_var, residuals, loadings_all, eigenvalues_df

    def fit_ou_process(self, series: pd.Series, window: int) -> Dict[str, float]:
        """Fit OU process to a residual series and return OU parameters."""
        if len(series) < window:
            return {'kappa': np.nan, 'm': np.nan, 'sigma': np.nan, 's_score': np.nan}
        X_t = series[-window:-1].values
        X_t_plus_1 = series[-window + 1:].values
        X_t = sm.add_constant(X_t)
        model = sm.OLS(X_t_plus_1, X_t).fit()
        b = model.params[1]
        if b <= 0 or b >= 1:
            return {'kappa': np.nan, 'm': np.nan, 'sigma': np.nan, 's_score': np.nan}
        kappa = -np.log(b)
        m = model.params[0] / (1 - b)
        sigma = np.sqrt(model.resid.var() * 2 * kappa / (1 - b**2))
        s_score = (series.iloc[-1] - m) / sigma if sigma != 0 else 0
        return {'kappa': kappa, 'm': m, 'sigma': sigma, 's_score': s_score}

    def apply_ou_fitting(self) -> pd.DataFrame:
        """Apply OU fitting to all residuals and return OU parameters DataFrame."""
        columns = pd.MultiIndex.from_product([self.residuals.columns, ['kappa', 'm', 'sigma', 's_score']])
        self.ou_params = pd.DataFrame(index=self.residuals.index, columns=columns)
        for t in range(self.ou_window, len(self.residuals)):
            date = self.residuals.index[t]
            for stock in self.residuals.columns:
                series = self.residuals[stock].iloc[:t + 1]
                params = self.fit_ou_process(series, self.ou_window)
                for param, value in params.items():
                    self.ou_params.loc[date, (stock, param)] = value
        return self.ou_params

    def generate_signals(self) -> pd.DataFrame:
        """Generate and return trading signals based on s-scores."""
        self.signals = pd.DataFrame(0, index=self.ou_params.index, columns=self.residuals.columns)
        for date in self.signals.index:
            for stock in self.signals.columns:
                s_score = self.ou_params.loc[date, (stock, 's_score')]
                if pd.notna(s_score):
                    if s_score < -1.25 and self.positions[stock] == 0:
                        self.signals.loc[date, stock] = 1  # Buy signal
                    elif s_score > -0.5 and self.positions[stock] > 0:
                        self.signals.loc[date, stock] = -1  # Sell signal
        return self.signals

    def open_stock_position(self, date: pd.Timestamp, stock: str, price: float) -> bool:
        """Attempt to open a stock position; return True if successful."""
        allocation = min(self.allocation_per_stock * self.cash, self.cash)
        shares = int(allocation / price)
        if shares <= 0:
            return False
        cost = shares * price * (1 + self.fee_rate)
        if cost <= self.cash:
            self.cash -= cost
            self.positions[stock] = shares
            self.entry_prices[stock] = price
            return True
        return False

    def close_stock_position(self, date: pd.Timestamp, stock: str, price: float) -> bool:
        """Attempt to close a stock position; return True if successful."""
        shares = self.positions[stock]
        if shares <= 0:
            return False
        proceeds = shares * price * (1 - self.fee_rate)
        self.cash += proceeds
        self.positions[stock] = 0
        self.entry_prices[stock] = 0
        return True

    def adjust_spy_short(self, date: pd.Timestamp, price_spy: float) -> bool:
        """Adjust SPY short position for beta neutrality; return True if adjusted."""
        V_s = sum(self.positions[stock] * self.data.loc[date, stock]
                  for stock in self.tickers if stock != 'SPY' and self.positions[stock] > 0)
        if V_s == 0:
            shares_short_desired = 0
        else:
            beta_s = sum((self.positions[stock] * self.data.loc[date, stock] / V_s) * self.betas.loc[date, stock]
                         for stock in self.tickers if stock != 'SPY' and self.positions[stock] > 0)
            V_short = beta_s * V_s
            shares_short_desired = int(V_short / price_spy)
        
        current_shares_short = -self.positions['SPY'] if self.positions['SPY'] < 0 else 0
        if shares_short_desired == current_shares_short:
            return False  # No adjustment needed

        # Close current SPY position if any
        if self.positions['SPY'] < 0:
            shares_to_close = -self.positions['SPY']
            proceeds = shares_to_close * price_spy * (1 - self.fee_rate)
            self.cash += proceeds + self.margin_posted
            self.margin_posted = 0
            self.positions['SPY'] = 0

        # Open new SPY short position
        if shares_short_desired > 0:
            margin_new = self.margin_spy * shares_short_desired * price_spy
            cost = shares_short_desired * price_spy * self.fee_rate
            if self.cash >= margin_new + cost:
                self.cash -= (margin_new + cost)
                self.margin_posted = margin_new
                self.positions['SPY'] = -shares_short_desired
                self.entry_prices['SPY'] = price_spy
                return True
        return False

    def run(self) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, Dict]:
        """Run the strategy and return prices, returns, asset DataFrame, and PCA metrics."""
        prices, returns = self.fetch_data()
        self.calculate_betas()
        explained_var, residuals, loadings, eigenvalues = self.compute_pca_factors()
        self.apply_ou_fitting()
        self.generate_signals()

        # Trading loop
        for t in range(max(self.pca_window, self.ou_window, self.beta_window), len(self.returns)):
            date = self.returns.index[t]
            if date not in self.signals.index:
                continue
            for stock in self.tickers:
                if stock == 'SPY':
                    continue
                signal = self.signals.loc[date, stock]
                price = self.data.loc[date, stock]
                if signal == 1:
                    self.open_stock_position(date, stock, price)
                elif signal == -1:
                    self.close_stock_position(date, stock, price)
            self.adjust_spy_short(date, self.data.loc[date, 'SPY'])
            # Record portfolio state
            for stock in self.tickers:
                if (stock != 'SPY' and self.positions[stock] > 0) or (stock == 'SPY' and self.positions['SPY'] < 0):
                    self.asset_list.append({
                        'date': date,
                        'ticker': stock,
                        'entry_price': self.entry_prices[stock],
                        'price_now': self.data.loc[date, stock],
                        'quantity': self.positions[stock],
                    })
        asset_df = pd.DataFrame(self.asset_list)
        self.pca_metrics = {
            'explained_variance': explained_var,
            'loadings': loadings,
            'eigenvalues': eigenvalues
        }
        return prices, returns, asset_df, self.pca_metrics


In [37]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN','META', 'TSLA', 'NVDA', 'AMD', 'INTC', 'CSCO',
           'XOM', 'CVX', 'COP', 'SLB', 'HAL', 'OXY', 'VLO', 'MPC', 'PSX', 'EOG',
           'JPM', 'BAC', 'WFC', 'C', 'GS', 'MS', 'PFE', 'JNJ', 'MRK', 'LLY',
           'PG', 'KO', 'PEP', 'WMT', 'TGT', 'COST', 'HD', 'LOW', 'DIS', 'NFLX',
           ]
strategy = PCA_StatArb_Strategy(tickers, start_date="2020-01-01", end_date="2022-12-31")
data,returns=strategy.fetch_data()

[*********************100%***********************]  41 of 41 completed


In [38]:
returns

Ticker,AAPL,AMD,AMZN,BAC,C,COP,COST,CSCO,CVX,DIS,...,PG,PSX,SLB,SPY,TGT,TSLA,VLO,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-03,-0.009722,-0.010183,-0.012139,-0.020763,-0.018836,0.003667,0.000824,-0.016316,-0.003459,-0.011471,...,-0.006725,-0.033066,0.009709,-0.007572,-0.010391,0.029633,-0.037948,-0.006139,-0.008828,-0.008040
2020-01-06,0.007968,-0.004321,0.014886,-0.001433,-0.003136,0.011872,0.000274,0.003569,-0.003388,-0.005802,...,0.001387,-0.006268,0.006410,0.003815,-0.009458,0.019255,0.000109,-0.005990,-0.002036,0.007678
2020-01-07,-0.004703,-0.002893,0.002092,-0.006600,-0.008685,0.000000,-0.001576,-0.006485,-0.012769,0.000343,...,-0.006192,0.003061,-0.005144,-0.002812,0.001780,0.038801,0.013110,-0.008286,-0.009265,-0.008184
2020-01-08,0.016086,-0.008705,-0.007809,0.010110,0.007618,-0.023165,0.011464,0.000632,-0.011423,-0.002059,...,0.004263,-0.037359,-0.029549,0.005329,-0.003231,0.049205,0.003882,0.003038,-0.003432,-0.015080
2020-01-09,0.021241,0.023834,0.004799,0.001716,0.009072,0.017401,0.016050,-0.004209,-0.001614,-0.003920,...,0.010938,0.014602,0.011418,0.006781,0.000811,-0.021945,0.022452,-0.001704,0.010330,0.007655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,-0.002798,0.010335,0.017425,0.002470,0.006138,0.043227,0.008589,0.003381,0.030916,0.015461,...,0.002825,0.033920,0.031134,0.005752,0.012734,-0.017551,0.039709,0.007375,0.002021,0.026445
2022-12-27,-0.013879,-0.019374,-0.025924,0.001848,0.002937,0.012132,-0.008970,0.001053,0.012571,-0.018634,...,0.008714,0.006157,0.009624,-0.003944,0.016766,-0.114089,0.016403,0.001464,0.000278,0.013894
2022-12-28,-0.030685,-0.011064,-0.014692,0.007378,0.005181,-0.026673,-0.012017,-0.009678,-0.014753,-0.025472,...,-0.012926,-0.023236,-0.016822,-0.012428,0.001718,0.033089,-0.008385,0.001949,-0.017523,-0.016426


In [39]:
betas=strategy.calculate_betas()
betas.head()

,AAPL,MSFT,GOOGL,AMZN,META,TSLA,NVDA,AMD,INTC,CSCO,...,PG,KO,PEP,WMT,TGT,COST,HD,LOW,DIS,NFLX
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-31,1.157080,1.148273,0.982607,0.701461,1.019642,1.278359,1.364578,1.185882,1.186278,1.010437,...,0.725945,0.836367,0.906100,0.521353,0.681327,0.609649,1.117427,1.216945,1.086587,0.667582
2021-01-04,1.158262,1.148759,0.983023,0.702604,1.020060,1.275127,1.361308,1.182988,1.184277,1.009219,...,0.725810,0.839850,0.908429,0.517961,0.678986,0.607871,1.116689,1.215100,1.086947,0.670269
2021-01-05,1.158358,1.148411,0.982472,0.702509,1.019674,1.274419,1.361954,1.182995,1.185085,1.008893,...,0.725917,0.839049,0.908245,0.517567,0.679856,0.607047,1.116689,1.214765,1.087029,0.669015
2021-01-06,1.156321,1.146662,0.981754,0.701173,1.018229,1.276070,1.359198,1.181302,1.184841,1.008988,...,0.726049,0.837231,0.907015,0.517400,0.681857,0.606127,1.116510,1.214901,1.086990,0.666487
2021-01-07,1.157910,1.147647,0.983562,0.701133,1.018697,1.281189,1.363894,1.186187,1.185608,1.008936,...,0.723504,0.834456,0.904934,0.516616,0.682181,0.603910,1.113449,1.212703,1.084882,0.666343


In [41]:
explained_var, residuals, loadings_all, eigenvalues_df=strategy.compute_pca_factors()

In [42]:
explained_var

Date
2020-12-31    0.939121
2021-01-04    0.938970
2021-01-05    0.939086
2021-01-06    0.939009
2021-01-07    0.938952
                ...   
2022-12-23    0.918962
2022-12-27    0.918827
2022-12-28    0.918970
2022-12-29    0.919282
2022-12-30         NaN
Length: 504, dtype: float64

In [43]:
loadings_all

,Factor_1,Factor_2,Factor_3,Factor_4,Factor_5,Factor_6,Factor_7,Factor_8,Factor_9,Factor_10,Factor_11,Factor_12,Factor_13,Factor_14,Factor_15,date
Ticker,,,,,,,,,,,,,,,,
AAPL,0.106017,-0.218422,-0.006925,0.057618,0.060352,0.062565,-0.134625,0.031183,-0.101992,-0.063247,-0.088454,-0.140797,-0.106876,-0.200953,0.136126,2020-12-31
AMD,0.110718,-0.282434,0.072732,0.086941,0.235542,-0.154022,-0.261190,-0.371418,0.629300,-0.070934,0.002918,-0.007294,-0.100044,0.213517,-0.085996,2020-12-31
AMZN,0.055986,-0.195937,0.069603,0.079232,0.118631,0.055100,-0.196042,-0.103108,-0.209648,0.128941,0.060592,-0.174030,-0.005909,0.001505,-0.053555,2020-12-31
BAC,0.186147,-0.000571,-0.156328,-0.028375,-0.297686,0.036995,-0.102769,-0.087354,0.049285,0.002131,0.038679,0.014259,-0.077079,-0.078848,-0.047700,2020-12-31
C,0.215308,0.014537,-0.118224,-0.117546,-0.296914,0.005713,-0.239415,0.038542,0.037826,-0.032226,0.102816,0.084451,-0.066865,0.067985,0.077090,2020-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSLA,0.261764,-0.181903,0.307022,0.603136,0.240934,-0.127567,0.251286,0.473620,-0.168411,-0.039486,-0.049167,0.100182,-0.034477,-0.105767,0.034246,2022-12-29
VLO,0.135873,0.274937,0.041178,0.012410,-0.142266,0.036118,-0.174458,0.016259,-0.304190,-0.444591,0.194570,0.115023,-0.016085,0.192543,-0.083069,2022-12-29
WFC,0.133997,-0.011988,-0.183233,-0.022282,-0.117705,-0.323526,0.162407,0.016964,0.043373,-0.043888,-0.032247,-0.018689,0.036965,0.156793,0.065684,2022-12-29


In [44]:
eigenvalues_df

,Eigen_1,Eigen_2,Eigen_3,Eigen_4,Eigen_5,Eigen_6,Eigen_7,Eigen_8,Eigen_9,Eigen_10,Eigen_11,Eigen_12,Eigen_13,Eigen_14,Eigen_15
Date,,,,,,,,,,,,,,,
2020-12-31,0.031332,0.006263,0.002808,0.001908,0.001464,0.001038,0.000861,0.000655,0.000635,0.000568,0.000502,0.000446,0.000398,0.000357,0.000326
2021-01-04,0.031327,0.006264,0.002802,0.001898,0.00146,0.001038,0.000861,0.000656,0.000639,0.000568,0.000502,0.000449,0.000399,0.000359,0.000328
2021-01-05,0.03143,0.006286,0.002802,0.001907,0.001461,0.001038,0.000861,0.000655,0.000636,0.000569,0.000501,0.000449,0.000399,0.000361,0.000328
2021-01-06,0.031479,0.006317,0.002801,0.001908,0.001514,0.001037,0.000863,0.000655,0.000637,0.000568,0.000507,0.000451,0.000399,0.00036,0.000334
2021-01-07,0.03152,0.006317,0.002811,0.001907,0.001509,0.001033,0.000866,0.000658,0.000637,0.000569,0.00051,0.000454,0.000403,0.000363,0.000336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,0.011422,0.004634,0.001428,0.001173,0.000962,0.000875,0.000585,0.000501,0.000489,0.000431,0.000389,0.000312,0.000282,0.000218,0.000204
2022-12-27,0.011396,0.004652,0.001439,0.00119,0.000962,0.00087,0.000585,0.000506,0.000488,0.000432,0.000388,0.000313,0.000282,0.000217,0.000202
2022-12-28,0.011411,0.004666,0.001439,0.001195,0.000963,0.000873,0.000586,0.000506,0.000488,0.000432,0.000389,0.000313,0.000283,0.000217,0.000202


In [ ]:
print(f"Annualized Return: {ann_return:.2%}, Sharpe Ratio: {sharpe:.2f}")
print("\nStock Returns Sample:")
print(returns_df.head())
print("\nStock Prices Sample:")
print(prices_df.head())
print("\nAsset DataFrame Sample:")
print(asset_df.head())